# Resources

https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf - The API user guide. Explains the proper way to format api calls and provides numerous examples.  

https://api.census.gov/data.html - The API discovery tool. This page lists every table that can be accessed via api. Each table comes with a treasure trove of information: the geographies, variables, and groups that can be accessed via a url stem that's provided. There are also example api calls that can be referenced when constructing calls for the specific information.

In [4]:
#Disable autosave
%autosave 0

Autosave disabled


In [2]:
#Import dependencies
from requests import get
from env import census_api_key
import pandas as pd

In [8]:
#Define the api url
url = f"https://api.census.gov/data/2019/acs/acs1/subject"

In [9]:
#Get a response
response = get(url)

In [10]:
#Extract the content in json format
content = response.json()

In [11]:
#Look at the keys
content.keys()

dict_keys(['@context', '@id', '@type', 'conformsTo', 'describedBy', 'dataset'])

In [16]:
#Look at the keys for datasets
content['dataset'][0].keys()

dict_keys(['c_vintage', 'c_dataset', 'c_geographyLink', 'c_variablesLink', 'c_tagsLink', 'c_examplesLink', 'c_groupsLink', 'c_sorts_url', 'c_valuesLink', 'c_documentationLink', 'c_isAggregate', 'c_isCube', 'c_isAvailable', '@type', 'title', 'accessLevel', 'bureauCode', 'description', 'distribution', 'contactPoint', 'identifier', 'keyword', 'license', 'modified', 'programCode', 'references', 'publisher'])

In [17]:
#Find new apis to hit
content['dataset'][0]['c_groupsLink']

'http://api.census.gov/data/2019/acs/acs1/subject/groups.json'

In [18]:
#Define new url
url_g = 'http://api.census.gov/data/2019/acs/acs1/subject/groups.json'

In [19]:
#Get response
response_g = get(url_g)

In [20]:
#Extract content
content_g = response_g.json()

In [21]:
#Check keys
content_g.keys()

dict_keys(['groups'])

In [25]:
#This api contains table descriptions and links!
content_g['groups'][0]

{'name': 'S0103PR',
 'description': 'POPULATION 65 YEARS AND OVER IN PUERTO RICO',
 'variables': 'http://api.census.gov/data/2019/acs/acs1/subject/groups/S0103PR.json'}

In [28]:
#Search for the specific table of interest
table = 'S1901'

link = ''

for item in content_g['groups']:
    
    if item['name'] == table:
        
        print('Found it: {}.'.format(item['name']))
        
        link = item['variables']
        
        break

Found it: S1901.


In [29]:
#Check the link
link

'http://api.census.gov/data/2019/acs/acs1/subject/groups/S1901.json'

In [30]:
#Define a new url to access the table
url_s1901 = 'https://api.census.gov/data/2019/acs/acs1/subject?get=NAME,group(S1901)&for=us:1&key=' + '{}'.format(census_api_key)

In [32]:
#Response
table_response = get(url_s1901)

In [33]:
#Content
table_content = table_response.json()

In [39]:
#Create a dataframe of the table
df_inc = pd.DataFrame(table_content)

In [40]:
#Look at the results
df_inc.head()

,0,1,2,3,4,5,6,7,8,9,...,250,251,252,253,254,255,256,257,258,259
0,NAME,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001EA,S1901_C01_001M,S1901_C01_001MA,S1901_C01_002E,S1901_C01_002EA,S1901_C01_002M,...,S1901_C04_014MA,S1901_C04_015E,S1901_C04_015EA,S1901_C04_015M,S1901_C04_015MA,S1901_C04_016E,S1901_C04_016EA,S1901_C04_016M,S1901_C04_016MA,us
1,United States,0100000US,United States,122802852,None,137327,None,5.8,None,0.1,...,(X),-888888888,(X),-888888888,(X),32.7,None,-888888888.0,(X),1


I am able to navigate their api to locate specific tables, however I am only able to retrieve results for the United States. I need to access data to a greater level of detail, specifically one 5-digit zip code. I am able to achieve this level of detail with some tables but not others. It appears that American Community Survey 1-year estimates don't offer the specificity I desire. I am able to drill down to individual zip codes when I access the 5-year estimates. I will start here.

In [3]:
url_5y = 'https://api.census.gov/data/2020/acs/acs5/subject?get=group(S1901)&for=zip%20code%20tabulation%20area:78209&key={}'.format(census_api_key)

In [5]:
response_5y = get(url_5y)
content_5y = response_5y.json()
pd.DataFrame(content_5y)

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001EA,S1901_C01_001M,S1901_C01_001MA,S1901_C01_002E,S1901_C01_002EA,S1901_C01_002M,S1901_C01_002MA,...,S1901_C04_014MA,S1901_C04_015E,S1901_C04_015EA,S1901_C04_015M,S1901_C04_015MA,S1901_C04_016E,S1901_C04_016EA,S1901_C04_016M,S1901_C04_016MA,zip code tabulation area
1,8600000US78209,ZCTA5 78209,18787,None,1114,None,5.5,None,1.6,None,...,(X),-888888888,(X),-888888888,(X),26.5,None,-888888888.0,(X),78209


Notice how I'm accessing the same table but my numbers are much smaller. Scrolling to the right reveals I retrieved information from zip code tabulation area 78209 (ZCTA5 78209 for short in column 1). My results are for the area of interest, but the column names are cryptic. Searching for the table name (S1901) on the API discovery tool reveals the meaning behind each variable name. I am not interested in collecting every bit of information from this table: I want to make a more targeted api call to retrieve information only relevant to households.

Example label: S1901_C01_001E  
S1901 - the name of the table  
C01 - first column  
001E - first row estimate. E = estimate, EA = annotated estimate, M = margin of error, MA = annotated margin of error.

In [7]:
df_5y = pd.DataFrame(content_5y)
df_5y.T

,0,1
0,GEO_ID,8600000US78209
1,NAME,ZCTA5 78209
2,S1901_C01_001E,18787
3,S1901_C01_001EA,None
4,S1901_C01_001M,1114
...,...,...
254,S1901_C04_016E,26.5
255,S1901_C04_016EA,None
256,S1901_C04_016M,-888888888.0
257,S1901_C04_016MA,(X)


In [9]:
df_5y = df_5y.T
df_5y.rename({0:'category', 1:'value'}, inplace=True)
df_5y.columns

Index(['category', 'value'], dtype='object')

In [12]:
df_5y_est = df_5y[df_5y['category'].str.endswith('E')]
df_5y_est

,category,value
value,NAME,ZCTA5 78209
2,S1901_C01_001E,18787
6,S1901_C01_002E,5.5
10,S1901_C01_003E,3.6
14,S1901_C01_004E,5.0
...,...,...
238,S1901_C04_012E,53029
242,S1901_C04_013E,83686
246,S1901_C04_014E,-888888888
250,S1901_C04_015E,-888888888


In [46]:
df_5y_house = df_5y_est[df_5y_est['category'].str.contains('C01')].copy()
df_5y_house

,category,value
2,S1901_C01_001E,18787
6,S1901_C01_002E,5.5
10,S1901_C01_003E,3.6
14,S1901_C01_004E,5.0
18,S1901_C01_005E,8.1
22,S1901_C01_006E,8.6
26,S1901_C01_007E,18.9
30,S1901_C01_008E,9.3
34,S1901_C01_009E,11.1
38,S1901_C01_010E,10.6


In [47]:
df_5y_house = df_5y_house.iloc[:-3]
df_5y_house

,category,value
2,S1901_C01_001E,18787
6,S1901_C01_002E,5.5
10,S1901_C01_003E,3.6
14,S1901_C01_004E,5.0
18,S1901_C01_005E,8.1
22,S1901_C01_006E,8.6
26,S1901_C01_007E,18.9
30,S1901_C01_008E,9.3
34,S1901_C01_009E,11.1
38,S1901_C01_010E,10.6


In [41]:
label_list = df_5y_house['category'].tolist()
label_list

['S1901_C01_001E',
 'S1901_C01_002E',
 'S1901_C01_003E',
 'S1901_C01_004E',
 'S1901_C01_005E',
 'S1901_C01_006E',
 'S1901_C01_007E',
 'S1901_C01_008E',
 'S1901_C01_009E',
 'S1901_C01_010E',
 'S1901_C01_011E',
 'S1901_C01_012E',
 'S1901_C01_013E']

In [36]:
label_url = 'https://api.census.gov/data/2020/acs/acs5/subject/variables/'

final_labels = []

for label in label_list:
    
    url = label_url + label + '.json'
    
    response = get(url)
    
    content = response.json()
    
    final_labels.append(content['label'].split('!!')[-1])
    
final_labels 

['Total',
 'Less than $10,000',
 '$10,000 to $14,999',
 '$15,000 to $24,999',
 '$25,000 to $34,999',
 '$35,000 to $49,999',
 '$50,000 to $74,999',
 '$75,000 to $99,999',
 '$100,000 to $149,999',
 '$150,000 to $199,999',
 '$200,000 or more',
 'Median income (dollars)',
 'Mean income (dollars)']

In [48]:
df_5y_house

,category,value
2,S1901_C01_001E,18787
6,S1901_C01_002E,5.5
10,S1901_C01_003E,3.6
14,S1901_C01_004E,5.0
18,S1901_C01_005E,8.1
22,S1901_C01_006E,8.6
26,S1901_C01_007E,18.9
30,S1901_C01_008E,9.3
34,S1901_C01_009E,11.1
38,S1901_C01_010E,10.6


In [49]:
df_5y_house.reset_index(inplace=True, drop=True)
df_5y_house

,category,value
0,S1901_C01_001E,18787
1,S1901_C01_002E,5.5
2,S1901_C01_003E,3.6
3,S1901_C01_004E,5.0
4,S1901_C01_005E,8.1
5,S1901_C01_006E,8.6
6,S1901_C01_007E,18.9
7,S1901_C01_008E,9.3
8,S1901_C01_009E,11.1
9,S1901_C01_010E,10.6


In [51]:
label_series = pd.Series(final_labels)

df_5y_label = pd.concat([df_5y_house, label_series], axis=1)

df_5y_label

,category,value,0
0,S1901_C01_001E,18787,Total
1,S1901_C01_002E,5.5,"Less than $10,000"
2,S1901_C01_003E,3.6,"$10,000 to $14,999"
3,S1901_C01_004E,5.0,"$15,000 to $24,999"
4,S1901_C01_005E,8.1,"$25,000 to $34,999"
5,S1901_C01_006E,8.6,"$35,000 to $49,999"
6,S1901_C01_007E,18.9,"$50,000 to $74,999"
7,S1901_C01_008E,9.3,"$75,000 to $99,999"
8,S1901_C01_009E,11.1,"$100,000 to $149,999"
9,S1901_C01_010E,10.6,"$150,000 to $199,999"


In [54]:
df_5y_label = df_5y_label[[0, 'value']]
df_5y_label

,0,value
0,Total,18787
1,"Less than $10,000",5.5
2,"$10,000 to $14,999",3.6
3,"$15,000 to $24,999",5.0
4,"$25,000 to $34,999",8.1
5,"$35,000 to $49,999",8.6
6,"$50,000 to $74,999",18.9
7,"$75,000 to $99,999",9.3
8,"$100,000 to $149,999",11.1
9,"$150,000 to $199,999",10.6


In [77]:
df_5y_label.rename(columns={0: 'label'}, inplace=True)

df_5y_label

/var/folders/xs/5tfyh3t955vfnnxmc19jtn680000gn/T/ipykernel_1144/431491333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5y_label.rename(columns={0: 'label'}, inplace=True)


,label,value
0,Total,18787
1,"Less than $10,000",5.5
2,"$10,000 to $14,999",3.6
3,"$15,000 to $24,999",5.0
4,"$25,000 to $34,999",8.1
5,"$35,000 to $49,999",8.6
6,"$50,000 to $74,999",18.9
7,"$75,000 to $99,999",9.3
8,"$100,000 to $149,999",11.1
9,"$150,000 to $199,999",10.6


Documenting the process:

It began with an api call to a specific table (S1901) to retrieve information for a certain zip code (78209). The format of the api call was:

'https://api.census.gov/data/2020/acs/acs5/subject?get=group(S1901)&for=zip%20code%20tabulation%20area:78209&key={}'.format(census_api_key)

The table is accessed by the group(S1901) call, and the zip code is spceified using the for=zip%20code%20tabulation%20area:78209 part of the url. This returned a very wide dataframe, with 250some columns and a couple rows. The data was easier to understand transposed. I used the naming convention described in the previous markdown cell to grab the estimates and return values from the first column of the table (C01). The first column contains the estimated income over the past 12 months for households in the area. I searched for the categories at https://api.census.gov/data/2020/acs/acs5/subject/variables.html.  

I clicked on one of them to reveal this page of information: https://api.census.gov/data/2020/acs/acs5/subject/variables/S1901_C01_001E.json.  

This api page contains the label associated with the specific category. I noticed the url was formulaic, so I created a list of all the categories and iterated through them, making requests of their unique pages and recording the associated labels. These labels were concatenated to the values and rearranged to form a proper dataframe.

In [55]:
s0101_url = 'https://api.census.gov/data/2020/acs/acs5/subject?get=group(S0101)&for=zip%20code%20tabulation%20area:78209&key={}'.format(census_api_key)

In [56]:
s0101_response = get(s0101_url)
s0101_content = s0101_response.json()
pd.DataFrame(s0101_content).T

,0,1
0,GEO_ID,8600000US78209
1,NAME,ZCTA5 78209
2,S0101_C01_001E,44706
3,S0101_C01_001EA,None
4,S0101_C01_001M,2157
...,...,...
910,S0101_C06_038E,-888888888
911,S0101_C06_038EA,(X)
912,S0101_C06_038M,-888888888
913,S0101_C06_038MA,(X)


In [57]:
s01_df = pd.DataFrame(s0101_content).T

In [61]:
s01_df.rename(columns={0: 'category', 1: 'value'}, inplace=True)

s01_df[s01_df['category'].str.endswith('E')]

,category,value
1,NAME,ZCTA5 78209
2,S0101_C01_001E,44706
6,S0101_C01_002E,2017
10,S0101_C01_003E,2488
14,S0101_C01_004E,3199
...,...,...
894,S0101_C06_034E,-888888888
898,S0101_C06_035E,-888888888
902,S0101_C06_036E,-888888888
906,S0101_C06_037E,-888888888


In [63]:
s01_final = s01_df[s01_df['category'].str.endswith('E')]

s01_final = s01_final[s01_final['category'].str.contains('C01')]

s01_final

,category,value
2,S0101_C01_001E,44706
6,S0101_C01_002E,2017
10,S0101_C01_003E,2488
14,S0101_C01_004E,3199
18,S0101_C01_005E,3624
22,S0101_C01_006E,3334
26,S0101_C01_007E,3412
30,S0101_C01_008E,2535
34,S0101_C01_009E,3158
38,S0101_C01_010E,2294


In [64]:
s01_label_url = 'https://api.census.gov/data/2020/acs/acs5/subject/variables/'

s01_label_list = s01_final['category'].tolist()

s01_final_labels = []

for label in s01_label_list:
    
    url = s01_label_url + label + '.json'
    
    response = get(url)
    
    content = response.json()
    
    s01_final_labels.append(content['label'].split('!!')[-1])
    
s01_final_labels 

['Total population',
 'Under 5 years',
 '5 to 9 years',
 '10 to 14 years',
 '15 to 19 years',
 '20 to 24 years',
 '25 to 29 years',
 '30 to 34 years',
 '35 to 39 years',
 '40 to 44 years',
 '45 to 49 years',
 '50 to 54 years',
 '55 to 59 years',
 '60 to 64 years',
 '65 to 69 years',
 '70 to 74 years',
 '75 to 79 years',
 '80 to 84 years',
 '85 years and over',
 '5 to 14 years',
 '15 to 17 years',
 'Under 18 years',
 '18 to 24 years',
 '15 to 44 years',
 '16 years and over',
 '18 years and over',
 '21 years and over',
 '60 years and over',
 '62 years and over',
 '65 years and over',
 '75 years and over',
 'Median age (years)',
 'Sex ratio (males per 100 females)',
 'Age dependency ratio',
 'Old-age dependency ratio',
 'Child dependency ratio',
 'Sex',
 'Age']

In [67]:
s01_final.reset_index(inplace=True, drop=True)
s01_final

,category,value
0,S0101_C01_001E,44706
1,S0101_C01_002E,2017
2,S0101_C01_003E,2488
3,S0101_C01_004E,3199
4,S0101_C01_005E,3624
5,S0101_C01_006E,3334
6,S0101_C01_007E,3412
7,S0101_C01_008E,2535
8,S0101_C01_009E,3158
9,S0101_C01_010E,2294


In [69]:
s01_final = pd.concat([s01_final, pd.Series(s01_final_labels)], axis=1)

In [76]:
s01_final.rename(columns={0: 'label'}, inplace=True)

s01_final = s01_final[['label', 'value']]

s01_final

,label,value
0,Total population,44706
1,Under 5 years,2017
2,5 to 9 years,2488
3,10 to 14 years,3199
4,15 to 19 years,3624
5,20 to 24 years,3334
6,25 to 29 years,3412
7,30 to 34 years,2535
8,35 to 39 years,3158
9,40 to 44 years,2294
